Dataset - https://www.kaggle.com/c/dogs-vs-cats/data

Source Code - https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/

In [1]:
# Data locations
training_set_first_class = "/data/dataset/training_set/dogs/"
training_set_second_class = "/data/dataset/training_set/cats/"

test_set_first_class = "/data/dataset/test_set/dogs/"
test_set_second_class = "/data/dataset/test_set/cats/"

The images need to be standarized so they'll have the same size.
Usually images are changes to small squares, here we are changing to 200*200 squares.
Smaller inputs mean a model that is faster to train (the size of the inputs is the number of pixels in the image).


In [2]:
from os import listdir
import imghdr
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

Using TensorFlow backend.


In [3]:
FIRST_CLASS_LABEL, SECOND_CLASS_LABEL = 0, 1

IMG_SIZE = (200, 200)

images_train, labels_train = [], []
images_test, labels_test = [], []

# Images loading
for folder in [training_set_first_class, training_set_second_class]:
    for file in listdir(folder):
        if file.endswith(".jpg"):
            image_full_path = folder + file

            # Determine class
            label = FIRST_CLASS_LABEL
            if folder == training_set_second_class:
                label = SECOND_CLASS_LABEL

            # Image loading
            image = load_img(image_full_path, target_size=IMG_SIZE)

            # Convert image to numpy array
            image = img_to_array(image)

            # Store
            images_train.append(image)
            labels_train.append(label)

for folder in [test_set_first_class, test_set_second_class]:
    for file in listdir(folder):
        if file.endswith(".jpg"):
            image_full_path = folder + file

            # Determine class
            label = FIRST_CLASS_LABEL
            if folder == test_set_second_class:
                label = SECOND_CLASS_LABEL

            # Image loading
            image = load_img(image_full_path, target_size=IMG_SIZE)

            # Convert image to numpy array
            image = img_to_array(image)

            # Store
            images_test.append(image)
            labels_test.append(label)

# Convert to a munpy arrays
images_train = asarray(images_train)
images_train /= 255
labels_train = asarray(labels_train)

images_test = asarray(images_test)
images_test /= 255
labels_test = asarray(labels_test)

# Saving the images and the labels
save('X_train', images_train)
save('y_train', labels_train)

save('X_test', images_test)
save('y_test', labels_test)

FileNotFoundError: [Errno 2] No such file or directory: '/data/dataset/training_set/dogs/'

Now, its time to build the CNN.
The architecture which is used here uses convolutions of 3*3
followed by a max-pooling layer.
Means that each layer is composed of:
Conv2D + MaxPooling

In [ ]:
import tensorflow as tf

def init_model(img_width, img_height):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(32, (3, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Conv2D(64, (3, 3)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(1))
    model.add(tf.keras.layers.Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    return model  

In [ ]:
from numpy import load

model = init_model(IMG_SIZE[0], IMG_SIZE[1])

X_train = load('X_train.npy')
y_train = load('y_train.npy')

BATCH_SIZE=64
EPOCHS=20

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

# model.save('trained_model.h5')

In [ ]:
from keras.models import load_model

# evaluate model:
# trained_model = load_model('trained_model.h5')

X_test = load('X_test.npy')
y_test = load('y_test.npy')

loss, metric_val = trained_model.evaluate(x=X_test, y=y_test)